# Optical Flow

Submitted by: [*name and ID*]


In [2]:
%matplotlib inline

import cv2
import numpy as np


###  Sparse optical flow

In this part you will calculate and visualize optical flow on a live video feed from a webcam.

First, find some good features to track, use either Harris corners, or Shi-Tomasi corners (`cv2.goodFeaturesToTrack`).

Then, you will track these features from frame to frame, and display their location on the live video feed. Use: `cv2.calcOpticalFlowPyrLK`.

As features tend to get lost during the time (check their returned status), it is advised to look for new features to track, from time to time (either every N frames, or when the features number falls below some level).

Try to add a visual trace of the motion. See for example: https://www.youtube.com/watch?v=E86NLzNbuL8


- API doc: https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html

- Example: http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html



In [3]:
cap = cv2.VideoCapture(0)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners =100,
                       qualityLevel = 0.1,
                       minDistance = 3,
                       blockSize = 10 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (20,20),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
#p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)


try:
    while True:
        ret,frame = cap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if ret:
            p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
            # calculate optical flow
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
            if len(p1)>0:
                
                # Select good points
                good_new = p1[st==1]
                good_old = p0[st==1]
                #print(len(good_new),len(good_old))
                # Create a mask image for drawing purposes
                mask = np.zeros_like(old_frame)

                # draw the tracks
                for i,(new,old) in enumerate(zip(good_new,good_old)):
                    a,b = new.ravel()
                    c,d = old.ravel()
                    mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
                    frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
                img = cv2.add(frame,mask)
            
                
        cv2.imshow('frame',img)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)


    cv2.destroyAllWindows()
    cap.release()
except:
    cap.release()

### Interactive tracking

In this part, we will add some interactive aspects to your previous implementation. 

First, the live video is shown without any features. 

Then, the user draws a region-of-interest (ROI) box on the video. Only the features that fall inside this region should be tracked from frame to frame. 

In every frame, plot the bounding box of the tracked features (using cv2.boundingRect).

Once again, since features tend to get lost over time, find some new features to track every now and then.

See for example: https://www.youtube.com/watch?v=kcrGkD2HOZs

Tutorial on handling mouse events in OpenCV: 
- http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_mouse_handling/py_mouse_handling.html



In [7]:
def optical_flow(image_clone,frame):

    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners =100,
                           qualityLevel = 0.1,
                           minDistance = 3,
                           blockSize = 10 )

    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (20,20),
                      maxLevel = 2,
                      criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Create some random colors
    color = np.random.randint(0,255,(100,3))

    # Take first frame and find corners in it
    #ret, old_frame = cap.read()
    old_frame= image_clone
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    #p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)


    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if ret:
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        if len(p1)>0:

            # Select good points
            good_new = p1[st==1]
            good_old = p0[st==1]
            #print(len(good_new),len(good_old))
            # Create a mask image for drawing purposes
            mask = np.zeros_like(old_frame)

            # draw the tracks
            for i,(new,old) in enumerate(zip(good_new,good_old)):
                a,b = new.ravel()
                c,d = old.ravel()
                mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
                frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
            img = cv2.add(frame,mask)



    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    return img

In [8]:
rect = (0,0,0,0)
startPoint = False
endPoint = False
def on_mouse(event,x,y,flags,params):

    global rect,startPoint,endPoint

    # get mouse click
    if event == cv2.EVENT_LBUTTONDOWN:
 
        if startPoint == True and endPoint == True:
            startPoint = False
            endPoint = False
            rect = (0, 0, 0, 0)

        if startPoint == False:
            rect = (x, y, 0, 0)
            startPoint = True
        elif endPoint == False:
            rect = (rect[0], rect[1], x, y)
            endPoint = True

cap = cv2.VideoCapture(0)
waitTime = 50

#Reading the first frame
(grabbed, frame) = cap.read()

while(cap.isOpened()):

    (grabbed, frame) = cap.read()

    cv2.namedWindow('frame')
    cv2.setMouseCallback('frame', on_mouse)    

    #drawing rectangle
    if startPoint == True and endPoint == True:
        x = rect[0]
        y = rect[1]
        w = rect[2]
        h = rect[3]
        #print(x,y,h,w)
        cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (255, 0, 255), 2)
        roi = frame[y:h,x:w]
        clone = roi.copy()
        imc = optical_flow(clone,roi)
        
        
        #cv2.imshow('c',imc)
    cv2.imshow('frame',frame)
    
    key = cv2.waitKey(waitTime) 

    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [5]:
# cv2.destroyAllWindows()
cap.release()